# CS35L-Lab 8: Dynamic Linking

## Step 1

We first created a C source file in the SEASnet server that computes `cos(sqrt(3.0))` and then prints it using `printf`.

Here is the source code of the program:

In [32]:
WORKING_DIR="~/CS35L/assign8"
SERVER_ADD="classtzh@lnxsrv06.seas.ucla.edu"

In [33]:
ssh -X $SERVER_ADD "cd $WORKING_DIR && cat mathComp.c"

#include <stdio.h>
#include <math.h>

int main() {
  printf("%.17g\n", cos(sqrt(3.0)));
  return 0;
}

Now we can compile it.

In [34]:
ssh -X $SERVER_ADD "cd $WORKING_DIR && gcc mathComp.c -o mathComp"

## Step 2

We will then check the dynamic libraries this program uses in SEASnet server.

In [35]:
ssh -X $SERVER_ADD "cd $WORKING_DIR && ldd ./mathComp"

	linux-vdso.so.1 =>  (0x00007ffcbabc1000)
	libc.so.6 => /lib64/libc.so.6 (0x00007fa12da7e000)
	/lib64/ld-linux-x86-64.so.2 (0x0000562ded003000)


## Step 3

In addition, we would like to see what system calls used by this program. `strace` will help us achieve this goal.

In [36]:
ssh -X $SERVER_ADD "cd $WORKING_DIR && strace ./mathComp"

execve("./mathComp", ["./mathComp"], [/* 15 vars */]) = 0
brk(NULL)                               = 0x21f1000
mmap(NULL, 4096, PROT_READ|PROT_WRITE, MAP_PRIVATE|MAP_ANONYMOUS, -1, 0) = 0x7fe938af4000
access("/etc/ld.so.preload", R_OK)      = -1 ENOENT (No such file or directory)
open("/etc/ld.so.cache", O_RDONLY|O_CLOEXEC) = 3
fstat(3, {st_mode=S_IFREG|0644, st_size=154224, ...}) = 0
mmap(NULL, 154224, PROT_READ, MAP_PRIVATE, 3, 0) = 0x7fe938ace000
close(3)                                = 0
open("/lib64/libc.so.6", O_RDONLY|O_CLOEXEC) = 3
read(3, "\177ELF\2\1\1\3\0\0\0\0\0\0\0\0\3\0>\0\1\0\0\0\20\35\2\0\0\0\0\0"..., 832) = 832
fstat(3, {st_mode=S_IFREG|0755, st_size=2127336, ...}) = 0
mmap(NULL, 3940800, PROT_READ|PROT_EXEC, MAP_PRIVATE|MAP_DENYWRITE, 3, 0) = 0x7fe938511000
mprotect(0x7fe9386c9000, 2097152, PROT_NONE) = 0
mmap(0x7fe9388c9000, 24576, PROT_READ|PROT_WRITE, MAP_PRIVATE|MAP_FIXED|MAP_DENYWRITE, 3, 0x1b8000) = 0x7fe9388c9000
mmap(0x7fe9388cf000, 16832, PROT_READ|PROT_WRITE

Because we know that dynamic libraries have suffix `.so`, to get all the system calls related to the dynamic libraries, we can simply `grep .so` in the output of the `strace`.

In [39]:
ssh -X $SERVER_ADD "cd $WORKING_DIR && strace ./mathComp 2>&1 | grep .so"

access("/etc/ld.so.preload", R_OK)      = -1 ENOENT (No such file or directory)
open("/etc/ld.so.cache", O_RDONLY|O_CLOEXEC) = 3
open("/lib64/libc.so.6", O_RDONLY|O_CLOEXEC) = 3


Because both opened files are closed later, we will also count the close system call as part of the dynamic linking process.

In [40]:
ssh -X $SERVER_ADD "cd $WORKING_DIR && strace ./mathComp 2>&1 | grep close"

close(3)                                = 0
close(3)                                = 0


These system calls are related to dynamic linking because the programs need to open and get access to the codes stored in these shared libraries in the runtime. When the programs do not need them anymore, it closes them.

## Step 4

We will then investigate a dozen or so commands in `/usr/bin` directory.

In [43]:
ssh -X $SERVER_ADD "cd $WORKING_DIR && ./lddBin.sh"

which: no smbpasswd in (/u/eng/class/classtzh/miniconda3/bin:/usr/local/bin:/usr/bin)
/usr/bin/Mail:
	linux-vdso.so.1 =>  (0x00007ffc2fd6d000)
	libgssapi_krb5.so.2 => /lib64/libgssapi_krb5.so.2 (0x00007f2820fd6000)
	libsmime3.so => /lib64/libsmime3.so (0x00007f2820daf000)
	libnss3.so => /lib64/libnss3.so (0x00007f2820a84000)
	libssl3.so => /lib64/libssl3.so (0x00007f2820838000)
	libnspr4.so => /lib64/libnspr4.so (0x00007f28205fa000)
	libplc4.so => /lib64/libplc4.so (0x00007f28203f4000)
	libc.so.6 => /lib64/libc.so.6 (0x00007f2820031000)
	libkrb5.so.3 => /lib64/libkrb5.so.3 (0x00007f281fd49000)
	libk5crypto.so.3 => /lib64/libk5crypto.so.3 (0x00007f281fb15000)
	libcom_err.so.2 => /lib64/libcom_err.so.2 (0x00007f281f911000)
	libkrb5support.so.0 => /lib64/libkrb5support.so.0 (0x00007f281f703000)
	libdl.so.2 => /lib64/libdl.so.2 (0x00007f281f4fe000)
	libkeyutils.so.1 => /lib64/libkeyutils.so.1 (0x00007f281f2fa000)
	libresolv.so.2 => /lib64/libresolv.so.2 (0x00007f281f0e0000)
	libnssutil3.so

Instead of looking through the entire output to look for error message, we will run the program again and this time only output the error message.

In [48]:
ssh -X $SERVER_ADD "cd $WORKING_DIR && ./lddBin.sh 2>&1 >/dev/null"

which: no smbpasswd in (/u/eng/class/classtzh/miniconda3/bin:/usr/local/bin:/usr/bin)


This error appears because there is a missing package called `smbpasswd`.

## Step 6

Now we will go ahead and get a sorted list of every dynamic library that is used by any of the commands on the list.

In [49]:
ssh -X $SERVER_ADD "cd $WORKING_DIR && ./lddBin.sh 2> /dev/null | grep so | sort -u"

	/lib64/ld-linux-x86-64.so.2 (0x00005577ee251000)
	/lib64/ld-linux-x86-64.so.2 (0x00005577fdbab000)
	/lib64/ld-linux-x86-64.so.2 (0x000055876394e000)
	/lib64/ld-linux-x86-64.so.2 (0x0000559cd34de000)
	/lib64/ld-linux-x86-64.so.2 (0x0000559e4a97b000)
	/lib64/ld-linux-x86-64.so.2 (0x000055a643a15000)
	/lib64/ld-linux-x86-64.so.2 (0x000055b88bc08000)
	/lib64/ld-linux-x86-64.so.2 (0x000055cabf96d000)
	/lib64/ld-linux-x86-64.so.2 (0x000055dc2ffb5000)
	/lib64/ld-linux-x86-64.so.2 (0x000055e4fd4a4000)
	/lib64/ld-linux-x86-64.so.2 (0x000055e983e0f000)
	/lib64/ld-linux-x86-64.so.2 (0x000055ec9c25f000)
	/lib64/ld-linux-x86-64.so.2 (0x000055f05ee35000)
	/lib64/ld-linux-x86-64.so.2 (0x000055f5ddb73000)
	/lib64/ld-linux-x86-64.so.2 (0x000055f67d713000)
	/lib64/ld-linux-x86-64.so.2 (0x000055f9d220c000)
	/lib64/ld-linux-x86-64.so.2 (0x00005600c4838000)
	/lib64/ld-linux-x86-64.so.2 (0x000056060a7cb000)
	/lib64/ld-linux-x86-64.so.2 (0x00005607913d4000)
	/lib64/ld-linux-x86-64.so.2 (0x0000560ff9398000)
